In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from collections import Counter

In [2]:
mainDF = pd.read_excel("Data//September_09//LabHoursData.xlsx")
mainDF.columns

Index(['#', 'Machine Type', 'Machine', 'Wing', 'Floor', 'Run No', 'Pcs',
       'StartDate', 'StartTime', 'StopDate', 'StopTime', 'Run %',
       'LateStartReason', 'Stop Reasons', 'Recipe', 'Run Intensity',
       'Ini Category', 'Grown Category', 'RunStatus', 'IniAvgHeight',
       'ProdHeight', 'GrownHeight', 'GrowingHours', 'BreakdownTime',
       'From To Hours', 'Actual Hours', 'RunHours', 'IniWeight', 'FinalWeight',
       'GrownWeight', 'Width', 'Length', 'Dmtr', 'Unit', 'Month', 'GrRate'],
      dtype='object')

In [3]:
#remove = "BreakdownTime", "GrowingHours"
final_DF = mainDF[["Machine Type", "Machine", "GrownWeight", "ProdHeight", "Dmtr", "GrowingHours",
                   "Actual Hours", "GrRate", "Stop Reasons", "LateStartReason"]]
#final_DF[final_DF.isna().any(axis=1)]
final_DF = final_DF.rename(columns = {'Machine Type':'Machine_Type', 'Stop Reasons':'Stop_Reasons', 'Actual Hours':'Actual_Hours'})

final_DF.tail(2)

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,GrowingHours,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
4888,New,AG47,58.4575,2.296425,44.34,144.15,215.0,12.56,"TIDE O , N/A",HLD CLN
4889,NaN,NaN,NaN,NaN,NaN,"Sum: 603,628.46",NaN,Avg: 12.33,NaN,NaN


In [4]:
nullID = final_DF[final_DF[['Machine_Type','Machine','GrownWeight', 'ProdHeight','Dmtr', \
                            'Actual_Hours', 'GrRate']].isna().any(axis=1)].index
nullID

Int64Index([4889], dtype='int64')

In [5]:
final_DF = final_DF.drop(nullID)

In [6]:
final_DF.head(2)

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,GrowingHours,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
0,Old,CB9,48.6192,2.177466,43.56,104.17,186.0,15.32,"TIDE O , V LEAK",ON TIME
1,Old,CB9,90.1550,3.339166,44.79,183.17,183.1,15.34,"TIDE O , N/A",ON TIME


In [7]:
def count_words(df_temp,column_name):
    df_temp[column_name] = df_temp[column_name].str.rstrip('')
    df_temp[column_name] = df_temp[column_name].str.lstrip('')

    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(',,',',')

    df_temp[column_name] = df_temp[column_name].str.replace('\' ','\'')
    df_temp[column_name] = df_temp[column_name].str.replace(' \'','\'')

    df_temp[column_name] = df_temp[column_name].str.rstrip(',')
    df_temp[column_name] = df_temp[column_name].str.lstrip(',')

    df_temp[column_name] = df_temp[column_name].str.split(',')
    df_temp[column_name] = df_temp[column_name].apply(lambda x: Counter(x))
    
    df_temp[column_name] = df_temp[column_name].apply(lambda x: dict(x))
    
    return df_temp

In [8]:
#join all string values of Stop_Reasons in groupby
df_Stop_reason = final_DF.groupby(['Machine','Machine_Type'])['Stop_Reasons'].agg(lambda x: ','.join(x.dropna())).reset_index()

#join all string values of LateStartReason in groupby
df_LateStartReason = final_DF.groupby(['Machine','Machine_Type'])['LateStartReason'].agg(lambda x: ','.join(x.dropna())).reset_index()

In [9]:
final_DF.columns

Index(['Machine_Type', 'Machine', 'GrownWeight', 'ProdHeight', 'Dmtr',
       'GrowingHours', 'Actual_Hours', 'GrRate', 'Stop_Reasons',
       'LateStartReason'],
      dtype='object')

In [10]:
df_Stop_reason['Stop_Reasons_counts'] = df_Stop_reason['Stop_Reasons']
df_Stop_reason = count_words(df_Stop_reason,'Stop_Reasons_counts')

df_LateStartReason['LateStartReason_counts'] = df_LateStartReason['LateStartReason']
df_LateStartReason = count_words(df_LateStartReason,'LateStartReason_counts')

In [11]:
final_DF.GrownWeight = pd.to_numeric(final_DF.GrownWeight)
final_DF.GrowingHours = pd.to_numeric(final_DF.GrowingHours)
final_DF.ProdHeight = pd.to_numeric(final_DF.ProdHeight)
final_DF.Dmtr = pd.to_numeric(final_DF.Dmtr)
final_DF.Actual_Hours = pd.to_numeric(final_DF.Actual_Hours)
final_DF.GrRate = pd.to_numeric(final_DF.GrRate)

In [12]:
df_machine_less35 = pd.merge(
        final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight','GrowingHours'].sum().reset_index(),
        final_DF.groupby(['Machine', 'Machine_Type'])['GrRate', 'Actual_Hours', 'ProdHeight','Dmtr'].mean().reset_index(), 
                                      how="left", on=["Machine", "Machine_Type"])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_580\1619885743.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight','GrowingHours'].sum().reset_index(),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_580\1619885743.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine', 'Machine_Type'])['GrRate', 'Actual_Hours', 'ProdHeight','Dmtr'].mean().reset_index(),


In [13]:
df_machine_less35.head(2)

,Machine,Machine_Type,GrownWeight,GrowingHours,GrRate,Actual_Hours,ProdHeight,Dmtr
0,AA1,Old,222.58900,566.90,12.675,193.450000,3.362735,44.912500
1,AA10,Old,191.33677,542.74,13.245,105.608333,2.854769,42.023333


In [14]:
# df_machine_less35.index.name = "ID"
# df_machine_less35.reset_index(inplace=True)

In [15]:
df_machine_less35.head(2)

,Machine,Machine_Type,GrownWeight,GrowingHours,GrRate,Actual_Hours,ProdHeight,Dmtr
0,AA1,Old,222.58900,566.90,12.675,193.450000,3.362735,44.912500
1,AA10,Old,191.33677,542.74,13.245,105.608333,2.854769,42.023333


In [16]:
df_machine_less35["GrownWeight_less_35"] = df_machine_less35['GrownWeight'] - (df_machine_less35["GrownWeight"] * 35 / 100)
df_machine_less35["GrRate_rank"] = df_machine_less35['GrRate'].apply(lambda x: 1 if (x>12.50 and x<=13.00 ) else 0)

In [17]:
#=============================================================================

In [18]:
#==============================================================================

In [19]:
def add_sorting_rank_to_column(temp_df, column_name, ascending_order):
    temp_df[column_name] = pd.to_numeric(temp_df[column_name])
    temp_df = temp_df.sort_values(by=[column_name], ascending=ascending_order).reset_index()
    temp_df.index.name = column_name + "_rank"
    temp_df = temp_df.reset_index()
    temp_df[column_name + "_rank"] += 1
    temp_df.index.name = "delete_" + column_name
    return temp_df

In [20]:
df_machine_less35

,Machine,Machine_Type,GrownWeight,GrowingHours,GrRate,Actual_Hours,ProdHeight,Dmtr,GrownWeight_less_35,GrRate_rank
0,AA1,Old,222.589000,566.90,12.6750,193.450000,3.362735,44.912500,144.682850,1
1,AA10,Old,191.336770,542.74,13.2450,105.608333,2.854769,42.023333,124.368901,0
2,AA11,Old,190.674980,546.90,11.9780,126.310000,4.508238,42.900000,123.938737,0
3,AA12,Old,207.805630,541.40,12.6260,165.680000,2.807560,40.066000,135.073660,1
4,AA13,Old,220.316385,568.07,11.7000,190.325000,4.067300,44.770000,143.205650,0
...,...,...,...,...,...,...,...,...,...,...
1046,CC16,Old,299.613990,669.39,12.9600,175.700000,3.153092,45.774000,194.749094,1
1047,CC17,Old,318.039400,684.98,13.5620,164.120000,3.047184,46.180000,206.725610,0
1048,CC20,Old,302.204115,684.48,13.3700,187.900000,2.929648,45.514000,196.432675,0
1049,CC21,Old,319.141510,666.80,14.1080,146.160000,3.611014,45.270000,207.441982,0


In [21]:
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "GrownWeight", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "GrowingHours", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "Actual_Hours", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "ProdHeight", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "Dmtr", False)
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35 = df_machine_less35.drop(["delete_Dmtr","delete_ProdHeight","delete_Actual_Hours","delete_GrownWeight","delete_GrowingHours"], axis=1)

In [22]:
df_machine_less35

,Dmtr_rank,ProdHeight_rank,Actual_Hours_rank,GrowingHours_rank,GrownWeight_rank,index,Machine,Machine_Type,GrownWeight,GrowingHours,GrRate,Actual_Hours,ProdHeight,Dmtr,GrownWeight_less_35,GrRate_rank
0,1,711,804,738,1,334,AE56,Old,395.802920,544.33,13.110000,138.075000,2.812158,57.895000,257.271898,0
1,2,1024,1029,829,2,280,AD7,Old,378.667995,517.82,14.178571,98.607143,1.709085,56.554286,246.134197,0
2,3,494,944,802,3,328,AE50,Old,342.662140,526.31,13.476000,122.360000,3.237217,56.176000,222.730391,0
3,4,879,999,892,14,175,AC23,Old,321.669755,496.95,14.130000,110.936000,2.387821,55.996000,209.085341,0
4,5,980,957,701,19,304,AE29,Old,318.284910,553.64,13.688333,120.435000,2.058399,54.416667,206.885192,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,1047,429,693,917,952,110,AB34,Old,174.169505,491.15,13.180000,147.937500,3.359678,36.562500,113.210178,0
1047,1048,728,940,982,996,395,AF60,Old,146.856340,447.33,14.697500,122.650000,2.763391,36.082500,95.456621,0
1048,1049,866,1035,965,782,171,AC2,Old,203.314390,468.48,18.500000,91.616667,2.414311,36.018333,132.154354,0
1049,1050,634,853,929,901,399,AF64,Old,186.671160,487.42,14.805000,132.615000,2.974998,35.777500,121.336254,0


In [23]:
df_machine_less35['Total_rank'] = df_machine_less35['GrownWeight_rank'] + df_machine_less35['Dmtr_rank'] + \
    df_machine_less35['GrowingHours_rank'] + df_machine_less35['Actual_Hours_rank'] + df_machine_less35["ProdHeight_rank"]

In [24]:
df_results = pd.merge(pd.merge(df_machine_less35,df_Stop_reason,how="left", on=["Machine", "Machine_Type"]),df_LateStartReason,how="left", on=["Machine", "Machine_Type"])

In [25]:
df_results.columns

Index(['Dmtr_rank', 'ProdHeight_rank', 'Actual_Hours_rank',
       'GrowingHours_rank', 'GrownWeight_rank', 'index', 'Machine',
       'Machine_Type', 'GrownWeight', 'GrowingHours', 'GrRate', 'Actual_Hours',
       'ProdHeight', 'Dmtr', 'GrownWeight_less_35', 'GrRate_rank',
       'Total_rank', 'Stop_Reasons', 'Stop_Reasons_counts', 'LateStartReason',
       'LateStartReason_counts'],
      dtype='object')

In [26]:
df_results = df_results[['index','Machine_Type','Machine','GrownWeight','GrownWeight_less_35','Dmtr','Actual_Hours', 
                         'GrowingHours','ProdHeight','GrRate','GrownWeight_rank','Actual_Hours_rank', 'GrowingHours_rank',
                         'ProdHeight_rank','Dmtr_rank','GrRate_rank','Total_rank','Stop_Reasons_counts', 'LateStartReason_counts']]

In [27]:
df_results.head(2)

,index,Machine_Type,Machine,GrownWeight,GrownWeight_less_35,Dmtr,Actual_Hours,GrowingHours,ProdHeight,GrRate,GrownWeight_rank,Actual_Hours_rank,GrowingHours_rank,ProdHeight_rank,Dmtr_rank,GrRate_rank,Total_rank,Stop_Reasons_counts,LateStartReason_counts
0,334,Old,AE56,395.802920,257.271898,57.895000,138.075000,544.33,2.812158,13.110000,1,804,738,711,1,0,2255,"{'CDA': 1, 'N/A': 4, 'TIDE O': 2, 'HEAT': 1}","{'W LINE': 1, 'UNIT MAINTENANCE': 1, 'HLM LT':..."
1,280,Old,AD7,378.667995,246.134197,56.554286,98.607143,517.82,1.709085,14.178571,2,1029,829,1024,2,0,2886,"{'CDA': 1, 'N/A': 6, 'POWER TRIP': 1, 'HEAT': ...","{'W LINE': 1, 'UNIT MAINTENANCE': 1, 'ARC': 1,..."


In [28]:
df_results = df_results.sort_values(by=['index'])
df_results = df_results.set_index("index")
if df_results.index[0] == 0:
    df_results.index += 1

In [29]:
def rangeApply(k):
    if k%1 <= .50:
        return (str(int(k)+.00) +'-'+ str(int(k)+.50))
    else:
        return (str(int(k)+.50) +'-'+ str(int(k)+1.00))

In [30]:
df_results["GrRange"] = df_results.GrRate.apply(rangeApply)
df_results[['GrRate', "GrRange"]]

,GrRate,GrRange
index,,
1,12.6750,12.5-13.0
2,13.2450,13.0-13.5
3,11.9780,11.5-12.0
4,12.6260,12.5-13.0
5,11.7000,11.5-12.0
...,...,...
1047,12.9600,12.5-13.0
1048,13.5620,13.5-14.0
1049,13.3700,13.0-13.5


In [31]:
if os.path.exists("quantityReport.xlsx"):
    os.remove("quantityReport.xlsx")
    
df_results.to_excel("quantityReport.xlsx", header=1, index_label="ID")

In [32]:
df_results.Machine.nunique()

1028

In [33]:
m,t = 'AA24','Old'
final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)]

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,GrowingHours,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
3087,Old,AA24,28.181215,1.515431,45.45,67.42,72.35,13.50,"CDA , N/A",ON TIME
3088,Old,AA24,78.545000,5.082000,44.32,204.00,204.00,11.62,"TIDE O , N/A",CD
3089,Old,AA24,73.125000,4.725000,44.67,185.00,185.00,12.32,"TIDE O , N/A",ON TIME
3090,Old,AA24,39.468870,2.755266,44.01,102.07,180.00,11.89,"TIDE O , N/A",ON TIME


In [34]:
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrownWeight.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrowingHours.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Actual_Hours.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].ProdHeight.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Dmtr.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrRate.mean())

219.320085
558.49
160.3375
3.51942425
44.6125
12.3325
